In [1]:
## 下面是想要寫一個 指定檔案的資料 

In [7]:
import numpy as np
import sqlite3
import pandas as pd
import csv
from datetime import datetime, timedelta
import time
import datetime
import requests
from io import StringIO
import yfinance as yf
from datetime import datetime, timedelta

#抓取現在的時間
def take_time():
    from time import gmtime, strftime
    a=strftime("%Y%m%d", gmtime())
    return a[0:4],a[4:6],a[6:8]

In [26]:
#對應公司名稱
def search_name(name):
    company=[]
    df = pd.read_csv('公司/上市.csv')  
    a=df["有價證券代號及名稱"].tolist()
    df = pd.read_csv('公司/上櫃.csv')  
    b=df["有價證券代號及名稱"].tolist()
    df = pd.read_csv('公司/興櫃.csv')  
    c=df["有價證券代號及名稱"].tolist()
    try:
        for i in a:
            if i.__contains__(name) :
                company=i
                #print(i, " is containing")
        for i in b:
            if i.__contains__(name) :
                company=i
                #print(i, " is containing")
        for i in c:
            if i.__contains__(name) :
                company=i
                #print(i, " is containing")
        return company.replace("\u3000", " ")
    except:
        return name
    
def check_tw_two(name):
    company=[]
    df = pd.read_csv('公司/上市.csv')  
    a=df["有價證券代號及名稱"].tolist()
    df = pd.read_csv('公司/上櫃.csv')  
    b=df["有價證券代號及名稱"].tolist()

    try:
        for i in a:
            if i.__contains__(name) :
                company=i
                #print(i, " is containing")
                return name+".tw"
        for i in b:
            if i.__contains__(name) :
                company=i
                #print(i, " is containing")
                return name+".two"
    except:
        return name

In [27]:
## Open file   指定 月份的代號txt  拿出來
def take_month_predict2(filename):
    filename=str(filename)
    doc_name= filename +".txt"   #修改這邊就好了
    
    fp = open(doc_name, "r")
    line = fp.readline()
    month_predict=[]
    ## 用 while 逐行讀取檔案內容，直至檔案結尾
    while line:
    #    print (line)
        line=line.replace(" ","")
        month_predict.append(line.replace("\n",""))
        line = fp.readline()
    fp.close()
    return month_predict

In [28]:
#取得初始報價 ， 當天報價也可以用這個function


def get_start_price(year,month,day,stock_symbol):

    # 設定目標日期
    # 指定某個月的11號
    target_date = datetime(year=year, month=month, day=day)    
    
    # 獲取台積電股票
    # stock_symbol = "2330.tw"  # 台積電在 Yahoo Finance 的代號
    stock = yf.Ticker(stock_symbol)
        

    # 獲取指定日期的股票歷史資料
    # 使用 period="1d" 表示只要那一天的資料
    stock_history = stock.history(start=target_date.strftime("%Y-%m-%d"), end=(target_date + timedelta(days=1)).strftime("%Y-%m-%d"))

    
    
    # 確保獲得資料
    if not stock_history.empty:
        # 打印收盤價
        closing_price = stock_history['Close'][0]
        #print(stock_symbol,f"這支股票在 {target_date.strftime('%Y年%m月%d日')} 的收盤價為: {closing_price}")
        return round(closing_price,2),target_date.strftime("%Y-%m-%d")
    else:

        print("未找到指定日期的資料 往前一天")
        target_date = target_date + timedelta(days=-1)
        year_1 = target_date.year
        month_1 = target_date.month
        day_1 = target_date.day
        target_date = datetime(year=year_1, month=month_1, day=day_1) 
        stock_history = stock.history(start=target_date.strftime("%Y-%m-%d"), end=(target_date + timedelta(days=1)).strftime("%Y-%m-%d"))

        
        
#取得最後報價

def get_final_price(year,month,day,stock_symbol):

    # 獲取台積電股票
    #stock_symbol = "2330.tw"  # 台積電在 Yahoo Finance 的代號
    stock = yf.Ticker(stock_symbol)

    # 設定目標日期
    # 指定某個月的11號
    target_date = datetime(year=year, month=month, day=day)

    # 獲取指定日期的股票歷史資料
    # 使用 period="1d" 表示只要那一天的資料
    stock_history = stock.history(start=target_date.strftime("%Y-%m-%d"), end=(target_date + timedelta(days=1)).strftime("%Y-%m-%d"))

    # 確保獲得資料
    if not stock_history.empty:
        # 打印收盤價
        closing_price = stock_history['Close'][0]
        #print(stock_symbol,f"這支股票在 {target_date.strftime('%Y年%m月%d日')} 的收盤價為: {closing_price}")
        return round(closing_price,2),target_date.strftime("%Y-%m-%d")
    else:

        print("未找到指定日期的資料 往後一天")
        target_date = target_date + timedelta(days=1)
        year_1 = target_date.year
        month_1 = target_date.month
        day_1 = target_date.day
        get_final_price(year_1,month_1,day_1,stock_symbol)




## 指定日期 順便就是指定檔案名稱

In [29]:
year=2023
month=12
filename=str(year)+str(month)
predict_list=take_month_predict2(filename)

In [30]:
#這個只能用在過去回測
def get_information_2(stock_symbol,year,month,day):

    #得到所有資訊
    print(stock_symbol,year,month,day)
    #得到公司名字
    name=search_name(stock_symbol)
    print(name)
    
    #先得到初始價格跟時間
    start_price,start_date=get_start_price(int(year),int(month),11,check_tw_two(stock_symbol))
    
    #再得到目前的價格跟時間
    final_month=int(month)+1
    if final_month>12:
        final_month=1
    current_price,final_update=get_final_price(int(year),int(final_month),int(day),check_tw_two(stock_symbol))
    
    #得到報酬率
    now_return=(float(current_price)-float(start_price))/float(start_price)
    now_return= round(now_return*100,2)
    #print(now_return)
    if float(now_return)>0:
        types="+"
    else:
        types="-"
    print(name,start_price,start_date,current_price,final_update,now_return,types)
    return name,start_price,start_date,current_price,final_update,now_return,types

In [31]:
# 更新月營收策略  當月股價
# db =  sqlite3.connect('db.sqlite3')
# #db.execute("INSERT INTO type_data (tag)   VALUES ('{}')".format(data))
# db.execute("delete from basicCurrent")
# db.commit()
pk=1

#得到最後日期

#寫進資料庫
for i in range(len(predict_list)):
    name,start_price,start_date,current_price,final_update,now_return,types=get_information_2(predict_list[i],year,month,11)

    print(final_update,name,start_date,start_price,final_update,current_price,now_return,types)
over_date=final_update
#     db =  sqlite3.connect('db.sqlite3')
#     db.execute("INSERT INTO basicCurrent (id,final_update,stock_name,start_date,start_price,over_date,current_price,now_return,type)   VALUES ('{}','{}','{}','{}','{}','{}','{}','{}','{}')".format(pk,final_update,name,start_date,start_price,over_date,current_price,now_return,types))
#     db.commit()
#     db.close()
#     pk=pk+1

1722 2023 12 11
1722 台肥
1722 台肥 64.3 2023-12-11 51.17 2023-01-11 -20.42 -
2023-01-11 1722 台肥 2023-12-11 64.3 2023-01-11 51.17 -20.42 -
2388 2023 12 11
2388 威盛
2388 威盛 161.5 2023-12-11 69.17 2023-01-11 -57.17 -
2023-01-11 2388 威盛 2023-12-11 161.5 2023-01-11 69.17 -57.17 -
2801 2023 12 11
2801 彰銀
2801 彰銀 17.95 2023-12-11 16.56 2023-01-11 -7.74 -
2023-01-11 2801 彰銀 2023-12-11 17.95 2023-01-11 16.56 -7.74 -
2812 2023 12 11
2812 台中銀
2812 台中銀 16.45 2023-12-11 12.32 2023-01-11 -25.11 -
2023-01-11 2812 台中銀 2023-12-11 16.45 2023-01-11 12.32 -25.11 -
2845 2023 12 11
2845 遠東銀
2845 遠東銀 12.5 2023-12-11 11.2 2023-01-11 -10.4 -
2023-01-11 2845 遠東銀 2023-12-11 12.5 2023-01-11 11.2 -10.4 -
2880 2023 12 11
2880 華南金
2880 華南金 22.2 2023-12-11 22.57 2023-01-11 1.67 +
2023-01-11 2880 華南金 2023-12-11 22.2 2023-01-11 22.57 1.67 +
2884 2023 12 11
2884 玉山金
2884 玉山金 25.6 2023-12-11 22.95 2023-01-11 -10.35 -
2023-01-11 2884 玉山金 2023-12-11 25.6 2023-01-11 22.95 -10.35 -
2885 2023 12 11
2885 元大金
2885 元大金 26.55 2023-12